# Serializing
For ahlive to work, data must be serialized into a standard format.

### serializing classes
ahlive incorporates these classes to achieve this:

ahlive class | input data type | use cases
--- | --- | ---
`ah.Array` | `collections.Iterable` | for single, unlabeled 1D arrays
`ah.DataFrame` | `pd.DataFrame` | for multiple, labeled 1D arrays
`ah.Array2D` | `np.ndarray` | for single, unlabeled 2D arrays
`ah.Dataset` | `xr.Dataset` | for multiple, labeled 2D arrays

### ahlive.Array
`ah.Array` is the basic building block for building an animation in ahlive.

To instantiate, `xs` and `ys` are required positional arguments; they are, respectively, the horizontal and vertical coordinates of a single data point.

`label` is a common keyword argument for instantiation which yields a legend label in the output animation. Note, the legend will not show automatically if there is only a single item so `legend=True` must be specified.

In [ ]:
import ahlive as ah
x = [1, 2, 3]
y = [40, 50, 60]
ah_arr = ah.Array(x, y, label='Array', legend=True)
print(ah_arr)
ah_arr.render()

<div class="alert alert-info">

The desired type of plot can be specified; e.g. `chart='bar'`

</div>

### ahlive.DataFrame
`ah.DataFrame` builds off `ah.Array`, but utilize column names.

To instantiate, `df` is required on top of `xs` and `ys`. However, unlike `ah.Array`, column names from the `pd.DataFrame` are passed, i.e. not the actual data values-- likewise for `label`.

An advantage of using `ah.DataFrame` instead of `ah.Array` is automatic labeling from column names.

In [ ]:
import pandas as pd
import ahlive as ah
x = [1, 2, 3]
y = [40, 50, 60]
df = pd.DataFrame({'the_x_name': x, 'y_axis': y})
ah_df = ah.DataFrame(df, 'the_x_name', 'y_axis')
print(ah_df)
ah_df.render()

Another advantage is if the `label` column contains multiple unique values; the unique values will be grouped and joined separately.

In [ ]:
import pandas as pd
import ahlive as ah
x = [1, 2, 3, 4]
y = [40, 50, 60, 70]
labels = ['a', 'a', 'b', 'b']
df = pd.DataFrame({'the_x_name': x, 'y_axis': y, 'label': labels})
display(df)
ah_df = ah.DataFrame(df, 'the_x_name', 'y_axis', label='label')
print(ah_df)
ah_df.render()

<div class="alert alert-info">

`ah.DataFrame` expects a tidy format `df`:

1. Each variable forms a column.
2. Each observation forms a row.
3. Each type of observational unit forms a table.

If the input `pd.DataFrame` is not tidy, try using `pd.melt`. An example can be found [here](../tutorials/covid_19_time_series).

</div>

### ahlive.Array2D
`ah.Array2D` is much like `ah.Array` but for gridded, 2D arrays.

In addition to `xs` and `ys`, `ah.Array2D` requires `cs` which is a 2D array used for colormapping.

In [ ]:
import numpy as np
import ahlive as ah
x = np.arange(0, 3)
y = np.arange(3, 6)
c = np.random.random((2, len(y), len(x)))
ah_arr2d = ah.Array2D(x, y, c)
ah_arr2d.render()

### ahlive.Dataset
`ah.Dataset` is the `ah.Array2D` version of `ah.DataFrame`.

`ah.Dataset` requires a total of four positional arguments: `ds`, `xs`, `ys`, `cs`. Similar to `ah.DataFrame`, these arguments should be labels from the `xr.Dataset`, not actual arrays.

In [ ]:
import numpy as np
import xarray as xr
import ahlive as ah
x = np.arange(0, 3)
y = np.arange(3, 6)
c = np.random.random((2, len(y), len(x)))
ds = xr.Dataset()
ds['color_data'] = xr.DataArray(
    c, coords={'the_x_name': x, 'y_axis': y},
    dims=('time_dim', 'y_axis', 'the_x_name'))
ah_ds = ah.Dataset(ds, 'the_x_name', 'y_axis', 'color_data')
print(ah_ds)
ah_ds.render()

<div class="alert alert-warning">

If you have an `xr.Dataset` with 1D arrays for data variables, do not use `ah.Dataset`!

Instead, first convert to a `pd.DataFrame` using `ds.to_dataframe()` then use `ah.DataFrame` to serialize the data.

</div>

### ahlive.Data

The aforementioned classes all inherit from the internal class, `ah.Data`.

It's essentially a higher level `dict`. The `dict` is accessible through the `data` property.

In [ ]:
import ahlive as ah
ah.Array([0, 1, 2], [3, 4, 5]).data

The keys of the `dict` store the subplots' positions as a (row, col) `tuple`.

In [ ]:
import ahlive as ah
arr_dict = ah.Array([0, 1, 2], [3, 4, 5]).data
print(arr_dict.keys())

The values of the `dict` store the serialized input data as `xr.Dataset`s.

In [ ]:
import ahlive as ah
arr_dict = ah.Array([0, 1, 2], [3, 4, 5]).data
print(arr_dict.values())

To access individual `xr.Dataset`s, simply specify the subplot's position.

In [ ]:
import ahlive as ah
arr_dict = ah.Array([0, 1, 2], [3, 4, 5]).data
print(arr_dict[1, 1])

You can also manually edit the `xr.Dataset` after calling `finalize`.

In [ ]:
import ahlive as ah
arr_dict = ah.Array([0, 1, 2], [3, 4, 5]).finalize().data
for rowcol, ds in arr_dict.items():
    ds['x'] += 10
print(arr_dict)